<h1> Speech Rap Signing Classification - LSTM</h1>

Source: https://www.youtube.com/watch?v=4nXI0h2sq2I&list=PL-wATfeyAMNrtbkCNsLcpoAyBBRJZVlnf&index=19

Data source: see my SRS_classifier.ipynb notebook

In [15]:
import numpy as np
import json

<h2>Data Preperation</h2>

We use the same MFCC data drawn at 16000 samples per second as we generated for the multi-layer perceptron approach used previously (see SRS_classifier.ipynb).

Data structure:

$x.\texttt{shape}=(\text{num samples},\text{num time intervals}, \text{num MFCC variables})$

$x_i.\texttt{shape} = (\text{num time intervals}, \text{num mfcc variables}))$


$y.\texttt{shape}=(\text{num labels})$

$y_i=\text{label}$

where $\text{label} \in \{1,2,3\}$

In [16]:
#load data from the file
#split data into training and test
"""
load data from a json file with objects "mfcc" and "labels"
@param dataset_path: path to the json file
@return np array of mfcc data, np array of target values
"""
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        inputs=np.array(data["mfcc"])
        targets=np.array(data["labels"])
        return inputs,targets


<h3>Load the data</h3>
Load the data from the jason file and verify that the inputs are of the expected type, and the sample size is as expected, in this case: 480 samples of each label.


In [17]:
x,y = load_data("data.json")
print ("input shape", x.shape)

unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the targets:")
print(np.asarray((unique_elements, counts_elements)))

input shape (1440, 94, 13)
Frequency of unique values of the targets:
[[  1   2   3]
 [480 480 480]]


In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.1)

In [19]:
import tensorflow.keras as keras
"""
Creates a model of three pooled convolutional layers, followed by a single fully connected layer and an output layer.
@param input_shape: the structure of SINGLE value x_i (omits the first dimension of x which is the total number of samples.)
@return uncompiled keras model of the CNN.
"""
def build_model(input_shape,dropoutRate=0):
    model=keras.Sequential()
    
    #LSTM Layer 1
    #return_sequences = True: sequence to sequence layer, otherwise sequence to vector
    model.add(keras.layers.LSTM(units=64,activation='tanh',input_shape=input_shape,return_sequences=True))
    #LSTM Layer 2
    model.add(keras.layers.LSTM(units=64,activation='tanh',input_shape=input_shape,return_sequences=False))
    
    #pipe output into a dense layer. No need to flatten because sequence to vector
    model.add(keras.Layers.Dense(units=64,activation='relu'))
    model.add(keras.Layers.Dropout(dropoutRate))
    
    #output layer
    #a fully connected layer for classification
    NUMBEROFPOSSIBLEOUTPUTS=3
    model.add(keras.layers.Dense(NUMBEROFPOSSIBLEOUTPUTS,activation='softmax'))
    return model

In [20]:
#each sample has the shape (num samples,intervals,variables,channels=1)
input_shape=(x_train.shape[1], x_train.shape[2])
model = build_model(input_shape,dropoutRate=0.4)

model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [21]:
#Adam is a very very effecting sgd variant for deep learning
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
#put all the components together
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"]
              )

NameError: name 'model' is not defined

In [22]:
#stores the progression of several metrics as the model trains.
history = model.fit(x_train,y_train,validation_data=(x_validation,y_validation),epochs=50)

NameError: name 'model' is not defined